<div style="text-align: center; background-color: #0A6EBD; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 40px; font-size: 28px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Final Project - Introduction To Data Science @ FIT-HCMUS, VNU-HCM
</div>

<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 30px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 01 - Data collecting
</div>

## 1. Introduction

- This lab is used to analyze the data about Teamfight Tactics (an auto battler game developed by Riot Games) on the Vietnam server (VN2).
- The primary focus lies in analyzing data related to player rankings in the Challenger and Grandmaster tiers, obtained through the Riot Games API.
- Because it involves real-time data, it requires time to collect data.

In [1]:
import datetime

now = datetime.datetime.now()
formatted_time = now.strftime("%H:%M")
formatted_date = now.strftime("%d/%m/%Y")

print(f"Collect data time: {formatted_time}, {formatted_date}")

Collect data time: 00:09, 20/12/2023


## 2. Set-up environment

In [2]:
#import neccessary library
import requests
import numpy as np
import pandas as pd
import json

## 3. Collect Data

In [3]:
# API key that RIOT Games access to collect data
API_key = 'RGAPI-a4f97ffd-0a2e-4430-8ca4-63f3f79f336b'
REGIONS = 'VN2'
TIERS = ['Challenger', 'Grandmaster']

headers = {
    'X-Riot-Token': API_key,
}

### Collect basic data

In [4]:
BASE_URL = 'api.riotgames.com/tft/league/v1'

def collect_data(tiersCode):
    data_dict = {'summonerId': [], 
                 'summonerName': [], 
                 'leaguePoints': [],
                 'rank': [], 
                 'wins': [],
                 'losses': [],
                 'veteran': [],
                 'inactive': [],
                 'freshBlood': [],
                 'hotStreak': [],
                 'tier': []}
    
    for tier in tiersCode:
        URL = f"https://{REGIONS.lower()}.{BASE_URL}/{tier.lower()}?queue=RANKED_TFT"
        response = requests.get(URL, headers=headers)
            
        if response.status_code == 200:
            summoner_data = response.json()
            entries = summoner_data['entries']

            for entrie in entries:
                for value in entrie:
                    data_dict[value].append(entrie[value])

                data_dict['tier'].append(tier)
                
    data = pd.DataFrame({"Summoner Id": data_dict['summonerId'],
                         "Summoner Name": data_dict['summonerName'],
                         "Summoner Points": data_dict['leaguePoints'], 
                         "Summoner Rank": data_dict['rank'],
                         "Summoner Wins": data_dict['wins'],
                         "Summoner Losses": data_dict['losses'],
                         "Is Veteran?": data_dict['veteran'],
                         "Is Inactive?": data_dict['inactive'],
                         "Is Fresh Blood?": data_dict['freshBlood'],
                         "Is Hot Streak?": data_dict['hotStreak'],
                         "Summoner Tier": data_dict['tier']})
    
    return data

In [5]:
tft_df = collect_data(tiersCode=TIERS)
tft_df

,Summoner Id,Summoner Name,Summoner Points,Summoner Rank,Summoner Wins,Summoner Losses,Is Veteran?,Is Inactive?,Is Fresh Blood?,Is Hot Streak?,Summoner Tier
0,rY978LAxHRRBj8Y7Bf_QEMXiQZ8trR2GJW-NS9Rpsk72az...,HLG Victorious,602,I,122,87,False,False,True,False,Challenger
1,5zFlRLcJb7MgSFht8_a2c9XPSEU1_WKJsLLSdBcZdqtMs5...,mikej,703,I,231,185,False,False,True,False,Challenger
2,cePM-S7ZzR-HtHClRmVaw46tVSyXImYZ8t0JgeK04ulxOZ...,H2B DoubleA,704,I,122,83,False,False,True,False,Challenger
3,lKv7DpczM6V21FMjCfbwVzaDFkPu-v27vPfb6o4hBAQna2...,TikTokTop1Gaming,613,I,115,81,False,False,True,False,Challenger
4,TVO29WY8nCcAW0d6fFQR8ANek9PPxvGQmy9CRtrFjamTk0...,kobisao,540,I,82,45,False,False,True,False,Challenger
...,...,...,...,...,...,...,...,...,...,...,...
1345,BVy1vP9KkR4T_9An3PBd_1EIUlojARi3n31VAInYE8JfwB...,Winter of aespa,251,I,113,98,False,False,True,True,Grandmaster
1346,vGTYRgKoqE_NGzMUZJzMR_saRRSlEgDzPzZs4t1ruqrVU_...,đau kì óc,327,I,110,77,False,False,True,False,Grandmaster
1347,mK-AysRwNKr4YVSj-pwQnOCwcpOhT_wBRTT08iY_-iB04X...,Con Chaos Non,509,I,182,157,False,False,True,True,Grandmaster
1348,gzEI_Y5y5xJpEblQ6bBr7fbVL_Cd7nIlW5beFR7HvCsPhX...,Thế gian vội vã,314,I,92,68,False,False,True,False,Grandmaster


### Collect `gameName`, `tagLine` and `summonerLevel` by `Summoner ID`

In [6]:
BASE_URL_puuid = 'api.riotgames.com/tft/summoner/v1/summoners'
BASE_URL_GameName = 'api.riotgames.com/riot/account/v1/accounts/by-puuid'

def collect_puuid(df):
    summonerLevel = []
    gameName = []
    tagLine = []
    
    for index, summonerId in enumerate(df['Summoner Id']):
        URL_puuid = f"https://{REGIONS.lower()}.{BASE_URL_puuid}/{summonerId}"
        response1 = requests.get(URL_puuid, headers=headers)
        
        if response1.status_code == 200:
            summoner_info = response1.json()
            summonerLevel.append(summoner_info['summonerLevel'])
            
            URL_GameName = f"https://asia.{BASE_URL_GameName}/{summoner_info['puuid']}"
            response2 = requests.get(URL_GameName, headers=headers)
            
            if response2.status_code == 200:
                gameName_info = response2.json()
                gameName.append(gameName_info['gameName'])
                tagLine.append(gameName_info['tagLine'])
            else:
                gameName.append('')
                tagLine.append('')
        else:
            summonerLevel.append('')
            gameName.append('')
            tagLine.append('')
            
    data = pd.DataFrame({"Riot Games Name": gameName,
                         "Riot Games Tag Line": tagLine,
                         "Summoner Level": summonerLevel})
    
    return data

In [7]:
detail_data = collect_puuid(tft_df)

### Concat `summoner_data` into `tft_df`

In [8]:
tft_df = pd.concat([detail_data, tft_df], axis=1)
tft_df

,Riot Games Name,Riot Games Tag Line,Summoner Level,Summoner Id,Summoner Name,Summoner Points,Summoner Rank,Summoner Wins,Summoner Losses,Is Veteran?,Is Inactive?,Is Fresh Blood?,Is Hot Streak?,Summoner Tier
0,Victorious,HLG,762,rY978LAxHRRBj8Y7Bf_QEMXiQZ8trR2GJW-NS9Rpsk72az...,HLG Victorious,602,I,122,87,False,False,True,False,Challenger
1,mikej,VN2,221,5zFlRLcJb7MgSFht8_a2c9XPSEU1_WKJsLLSdBcZdqtMs5...,mikej,703,I,231,185,False,False,True,False,Challenger
2,Duy Pingboda,9999,27,cePM-S7ZzR-HtHClRmVaw46tVSyXImYZ8t0JgeK04ulxOZ...,H2B DoubleA,704,I,122,83,False,False,True,False,Challenger
3,Top1GamingTopTop,6789,1,lKv7DpczM6V21FMjCfbwVzaDFkPu-v27vPfb6o4hBAQna2...,TikTokTop1Gaming,613,I,115,81,False,False,True,False,Challenger
4,kobisao,VN2,380,TVO29WY8nCcAW0d6fFQR8ANek9PPxvGQmy9CRtrFjamTk0...,kobisao,540,I,82,45,False,False,True,False,Challenger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,Minjeongiee,0101,122,BVy1vP9KkR4T_9An3PBd_1EIUlojARi3n31VAInYE8JfwB...,Winter of aespa,251,I,113,98,False,False,True,True,Grandmaster
1346,sudangcap,dogz,19,vGTYRgKoqE_NGzMUZJzMR_saRRSlEgDzPzZs4t1ruqrVU_...,đau kì óc,327,I,110,77,False,False,True,False,Grandmaster
1347,COM Chaos,2807,61,mK-AysRwNKr4YVSj-pwQnOCwcpOhT_wBRTT08iY_-iB04X...,Con Chaos Non,509,I,182,157,False,False,True,True,Grandmaster
1348,Walrus,9915,124,gzEI_Y5y5xJpEblQ6bBr7fbVL_Cd7nIlW5beFR7HvCsPhX...,Thế gian vội vã,314,I,92,68,False,False,True,False,Grandmaster


## 4. Save collected data

In [9]:
save_name = 'Group25_TFT.csv'
tft_df.to_csv('../data/raw/' + save_name, index=False)